## Loading

In [1]:
import sys
 
sys.path.insert(0, "../")

In [7]:
from dataAnalysis.DataAnalysis import DataAnalysis
import pandas as pd
import cudf
import torch

data = pd.read_csv(r"../extdata/sbcdata.csv", header=0)
data_analysis = DataAnalysis(data)

/home/dwalke/git/sbc/statistics/../dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Training: 
Assessable data are 528101 cases and 1015074 CBCs
Control data are 527038 cases and 1013548 CBCs
Sepsis data are 1488 cases and 1526 CBCs
$$$$$$$$$$$$$$$$$$$$
Testing: 


/home/dwalke/git/sbc/statistics/../dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 365794, Sepsis: 490
Assessable data are 180494 cases and 366284 CBCs
Control data are 180157 cases and 365794 CBCs
Sepsis data are 472 cases and 490 CBCs


/home/dwalke/git/sbc/statistics/../dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 437629, Sepsis: 448
Assessable data are 157922 cases and 438077 CBCs
Control data are 180157 cases and 437629 CBCs
Sepsis data are 438 cases and 448 CBCs


## Sorting

In [8]:
data = pd.concat((data_analysis.get_training_data(), data_analysis.get_testing_data()))
max_Id = data["Id"].unique().max()
gw_data = data_analysis.get_gw_testing_data().copy(deep=True)
gw_data = gw_data.assign(Id=lambda x: x.Id + max_Id)
data = pd.concat((data, gw_data))
data = data.sort_values("Id")
unique_ids = data["Id"].unique()
data = data.reset_index(drop=True)
data.pop("index")
data = data.sort_values(["Id", "Time"])
data = data.reset_index(drop=True)

In [9]:
data

,Id,Age,Sex,Diagnosis,Center,Set,Sender,Episode,Time,TargetIcu,SecToIcu,CRP,HGB,MCV,PCT,PLT,RBC,WBC,Label
0,1,79,M,Control,Leipzig,Training,ED,1,420.0,NaN,NaN,0.75,8.1,86.0,NaN,167.0,4.36,7.30,Control
1,2,35,M,Control,Leipzig,Training,GEN,1,0.0,NaN,NaN,NaN,10.6,79.9,NaN,199.0,6.02,4.70,Control
2,3,47,W,Control,Leipzig,Training,AMB,1,0.0,NaN,NaN,3.87,8.7,89.9,NaN,298.0,4.37,7.10,Control
3,4,28,M,Control,Leipzig,Training,ED,1,780.0,NaN,NaN,0.52,9.9,96.2,NaN,216.0,4.79,5.30,Control
4,5,63,W,Control,Leipzig,Training,GEN,1,0.0,NaN,NaN,232.87,7.4,86.6,NaN,189.0,3.96,20.90,Control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1819430,948120,56,M,Control,Greifswald,Validation,GEN,1,118380.0,NaN,NaN,NaN,8.7,88.1,NaN,200.0,4.70,6.20,Control
1819431,948120,56,M,Control,Greifswald,Validation,GEN,1,168660.0,NaN,NaN,95.00,8.7,88.4,NaN,233.0,4.70,6.92,Control
1819432,948120,56,M,Control,Greifswald,Validation,GEN,1,340440.0,NaN,NaN,63.60,8.1,87.6,NaN,225.0,4.50,3.70,Control
1819433,948121,60,W,Control,Greifswald,Validation,ED,1,0.0,NaN,NaN,NaN,9.1,90.0,NaN,337.0,4.80,10.80,Control


## Filter

In [4]:
# import numpy as np
# data = data[np.logical_and(data["Set"] == "Validation", data["Center"] == "Leipzig")]

## Evaluating sepsis positions

In [10]:
sepsis_mask = data["Label"] == "Sepsis"
sepsis_data = data[sepsis_mask]
sepsis_ids = sepsis_data["Id"].unique()
data_containing_sepsis_mask = data["Id"].isin(sepsis_ids)
data_containing_sepsis = data[data_containing_sepsis_mask]
data_containing_sepsis = data_containing_sepsis.sort_values(["Id", "Time"])
data_containing_sepsis = data_containing_sepsis.reset_index(drop=True)

In [12]:
all_rel_sepsis_positions = []
number_of_single_sepsis_cbcs = 0
i = 0
for Id, group in data_containing_sepsis.groupby(["Id"]):
    group = group.reset_index(drop=True)
    sepsis_mask = group["Label"] == "Sepsis"
    sepsis_positions = group.index[sepsis_mask].to_numpy().tolist()
    if group.shape[0] == 1:
        number_of_single_sepsis_cbcs += 1
        continue
    sepsis_positions_rel = list(map(lambda pos: pos / (group.shape[0]-1), sepsis_positions))
    all_rel_sepsis_positions.extend(sepsis_positions_rel)        

/tmp/ipykernel_769768/682914744.py:4: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for Id, group in data_containing_sepsis.groupby(["Id"]):


In [13]:
import numpy as np
np.sort(np.asarray(all_rel_sepsis_positions))

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.33333333, 0.5       ,
       0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
       0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
       0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.66666667,
       0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.75      ,
       0.75      , 0.75      , 0.8       , 0.8       , 0.8       ,
       0.83333333, 0.83333333, 0.83333333, 0.85714286, 0.875     ,
       0.875     , 0.88888889, 0.88888889, 0.90909091, 0.92307692,
       0.92307692, 0.92857143, 0.96      , 0.96969697, 0.97297297,
       0.98      , 0.98484848, 0.98507463, 0.98611111, 0.98823529,
       0.99590164, 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.     

In [14]:
number_of_single_sepsis_cbcs

1624

In [26]:
rel_sepsis_pos = np.asarray(all_rel_sepsis_positions)
(rel_sepsis_pos == 1).sum() / rel_sepsis_pos.shape

array([0.92142857])